Установка необходимых библиотек


In [ ]:
!pip install fasttext
!pip install tqdm

Монтирование диска Google Drive для загрузк и харанения  данных.

**ВНИМАНИЕ** для доступа к данным необходимо выполнить следующие действия:

1. Перейти по [ссылке](https://drive.google.com/drive/folders/1MnoxVXjG8o8kL0SS14E3YzbZmCovsGpR?usp=sharing)
2. Создать ярлык на данную директорию в своём Google Drive. Для этого нажмите на название директории а затем на "Добавить ярлык на Диск".

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Импортирование нужных библиотек

In [ ]:
from fasttext import FastText
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

prefix = Path("/content/gdrive/My Drive/Kononov_NLP")

Выбор модели для получения эмбеддингов

In [ ]:
name, alias = "ft_native_300_ru_wiki_lenta_lower_case", "wiki"
# name, alias = "ft_native_300_ru_twitter_nltk_word_tokenize", "twitter"
# name, alias = "cc.ru.300", "original"

Загрузка предобученной модели с диска Google Drive

In [ ]:
model = FastText.load_model(prefix / "FastText/{}.bin".format(name))

Подсчёт близости запросов и загоовков документов в пространстве эмбеддингов.

In [ ]:
queries_df = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)
titles_df = pd.read_csv(prefix / "titles.tsv", sep="\t", index_col=0).fillna("")
relation_df = pd.read_csv(prefix / "samples.tsv", sep="\t")

relation_df.drop(columns=["label", "sample_id"], inplace=True)
relation_df = relation_df.assign(embedding_similarity=np.full(relation_df.index.size, 0.0))
for key, group in tqdm(relation_df.groupby("query_id"), position=0, leave=True):
  doc_idx = group["doc_id"].values
  query_embedding = model.get_sentence_vector(queries_df["text"].loc[key])
  title_embeddings = np.array([x for x in (titles_df["text"].loc[doc_idx]).apply(model.get_sentence_vector)])
  relation_df.loc[group.index, "embedding_similarity"] = cosine_similarity(query_embedding.reshape(1, -1),
                                                                           title_embeddings).reshape(-1)

path = prefix / "semantic/fasttext_{}_query_doc_embedding_similarity.tsv".format(alias)
relation_df.to_csv(path, sep="\t", index=False)
print("Result saved to file: {}".format(path))